# InitPlus 3D Visualization (RHGBlock)

- 3DでInitPlusブロックのノードとエッジを可視化します。
- データ: 白塗り、X ancilla: 緑、Z ancilla: 青、入力/出力: 黒塗り
- オプション: ancilla 表示切替（both/x/z）、エッジ太さ
- 可能ならインタラクティブ（回転/ズーム）を有効化


In [1]:
# Interactive plotting setup with plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
print('Using plotly for interactive 3D visualization')

ModuleNotFoundError: No module named 'plotly'

In [ ]:
# 可視化ユーティリティの定義
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from lspattern.template.base import RotatedPlanarTemplate
from lspattern.blocks.initialize import InitPlus

def visualize_initplus(d=3, kind=("X", "Z", "X"), ancilla_mode="both", edge_width=3.0):
    template = RotatedPlanarTemplate(d=d, kind=kind)
    _ = template.to_tiling()
    block = InitPlus(d=d, kind=kind, template=template)
    block.materialize()

    g = block.graph_local
    node2coord = block.node2coords
    node2role = block.node2role

    color_map = {
        'data':      {'face': 'white',   'edge': 'black',  'size': 40},
        'ancilla_x': {'face': '#2ecc71','edge': '#1e8449','size': 36},
        'ancilla_z': {'face': '#3498db','edge': '#1f618d','size': 36},
    }
    groups = {k: {'x': [], 'y': [], 'z': []} for k in color_map}
    for n, (x, y, z) in node2coord.items():
        role = node2role.get(n, 'data')
        if role not in groups:
            role = 'data'
        groups[role]['x'].append(x)
        groups[role]['y'].append(y)
        groups[role]['z'].append(z)

    fig = plt.figure(figsize=(8, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_box_aspect((1, 1, 1))
    ax.set_title(f'InitPlus RHGBlock d={d} kind={kind}')

    for role, pts in groups.items():
        if ancilla_mode == 'x' and role == 'ancilla_z':
            continue
        if ancilla_mode == 'z' and role == 'ancilla_x':
            continue
        if not pts['x']:
            continue
        spec = color_map[role]
        ax.scatter(pts['x'], pts['y'], pts['z'], s=spec['size'], c=spec['face'], edgecolors=spec['edge'], depthshade=True, label=role, linewidths=0.8)

    for u, v in g.physical_edges:
        x1, y1, z1 = node2coord[u]
        x2, y2, z2 = node2coord[v]
        ax.plot([x1, x2], [y1, y2], [z1, z2], color='black', linewidth=edge_width, alpha=0.9)

    in_nodes = set(g.input_node_indices.keys())
    out_nodes = set(g.output_node_indices.keys())
    if in_nodes:
        xin = [node2coord[n][0] for n in in_nodes]
        yin = [node2coord[n][1] for n in in_nodes]
        zin = [node2coord[n][2] for n in in_nodes]
        ax.scatter(xin, yin, zin, s=60, c='black', edgecolors='black', label='input', zorder=5)
    if out_nodes:
        xout = [node2coord[n][0] for n in out_nodes]
        yout = [node2coord[n][1] for n in out_nodes]
        zout = [node2coord[n][2] for n in out_nodes]
        ax.scatter(xout, yout, zout, s=60, c='black', edgecolors='black', label='output', zorder=6)

    ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
    ax.legend(loc='upper left')
    plt.tight_layout()
    return fig, ax


In [ ]:
# Interactive 3D visualization with plotly
from lspattern.template.base import RotatedPlanarTemplate
from lspattern.blocks.initialize import InitPlus

def create_interactive_initplus(d=3, kind=("X", "Z", "X"), ancilla_mode="both", edge_width=3.0, show_edges=True):
    """Create interactive 3D plotly visualization of InitPlus block"""
    template = RotatedPlanarTemplate(d=d, kind=kind)
    _ = template.to_tiling()
    block = InitPlus(d=d, kind=kind, template=template)
    block.materialize()

    g = block.graph_local
    node2coord = block.node2coords
    node2role = block.node2role

    # Color mapping
    color_map = {
        'data':      {'color': 'white',   'line_color': 'black',  'size': 8, 'name': 'Data'},
        'ancilla_x': {'color': '#2ecc71', 'line_color': '#1e8449', 'size': 7, 'name': 'X Ancilla'},
        'ancilla_z': {'color': '#3498db', 'line_color': '#1f618d', 'size': 7, 'name': 'Z Ancilla'},
    }
    
    fig = go.Figure()
    
    # Group nodes by role
    groups = {k: {'x': [], 'y': [], 'z': [], 'nodes': []} for k in color_map}
    for n, (x, y, z) in node2coord.items():
        role = node2role.get(n, 'data')
        if role not in groups:
            role = 'data'
        groups[role]['x'].append(x)
        groups[role]['y'].append(y)
        groups[role]['z'].append(z)
        groups[role]['nodes'].append(n)

    # Add node traces
    for role, pts in groups.items():
        if ancilla_mode == 'x' and role == 'ancilla_z':
            continue
        if ancilla_mode == 'z' and role == 'ancilla_x':
            continue
        if not pts['x']:
            continue
        
        spec = color_map[role]
        fig.add_trace(go.Scatter3d(
            x=pts['x'], y=pts['y'], z=pts['z'],
            mode='markers',
            marker=dict(
                size=spec['size'],
                color=spec['color'],
                line=dict(color=spec['line_color'], width=1),
                opacity=0.9
            ),
            name=spec['name'],
            text=[f"Node {n}: {role}" for n in pts['nodes']],
            hovertemplate="<b>%{text}</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>"
        ))

    # Add edges
    if show_edges:
        edge_x, edge_y, edge_z = [], [], []
        for u, v in g.physical_edges:
            x1, y1, z1 = node2coord[u]
            x2, y2, z2 = node2coord[v]
            edge_x.extend([x1, x2, None])
            edge_y.extend([y1, y2, None])
            edge_z.extend([z1, z2, None])
        
        fig.add_trace(go.Scatter3d(
            x=edge_x, y=edge_y, z=edge_z,
            mode='lines',
            line=dict(color='black', width=edge_width),
            name='Edges',
            showlegend=False,
            hoverinfo='none'
        ))

    # Add input/output nodes
    in_nodes = set(g.input_node_indices.keys())
    out_nodes = set(g.output_node_indices.keys())
    
    if in_nodes:
        xin = [node2coord[n][0] for n in in_nodes]
        yin = [node2coord[n][1] for n in in_nodes]
        zin = [node2coord[n][2] for n in in_nodes]
        fig.add_trace(go.Scatter3d(
            x=xin, y=yin, z=zin,
            mode='markers',
            marker=dict(size=10, color='red', symbol='diamond'),
            name='Input',
            text=[f"Input node {n}" for n in in_nodes],
            hovertemplate="<b>%{text}</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>"
        ))
    
    if out_nodes:
        xout = [node2coord[n][0] for n in out_nodes]
        yout = [node2coord[n][1] for n in out_nodes]
        zout = [node2coord[n][2] for n in out_nodes]
        fig.add_trace(go.Scatter3d(
            x=xout, y=yout, z=zout,
            mode='markers',
            marker=dict(size=10, color='darkred', symbol='diamond'),
            name='Output',
            text=[f"Output node {n}" for n in out_nodes],
            hovertemplate="<b>%{text}</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>"
        ))

    # Layout settings with reversed X and Y axes
    fig.update_layout(
        title=f'Interactive InitPlus RHGBlock (d={d}, kind={kind})',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            xaxis=dict(autorange='reversed'),
            yaxis=dict(autorange='reversed'),
            aspectmode='cube',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        width=800,
        height=600,
        margin=dict(l=0, r=0, b=0, t=40)
    )
    
    return fig

In [ ]:
# Interactive controls and visualization
def create_interactive_widget():
    """Create interactive widget controls for the visualization"""
    
    # Widget controls
    d_slider = widgets.IntSlider(
        value=3, min=2, max=7, step=1,
        description='Distance (d):', 
        style={'description_width': 'initial'}
    )
    
    kind_dropdown = widgets.Dropdown(
        options=[
            ('X-Z-X', ('X', 'Z', 'X')),
            ('Z-X-Z', ('Z', 'X', 'Z')),
            ('X-X-X', ('X', 'X', 'X')),
            ('Z-Z-Z', ('Z', 'Z', 'Z'))
        ],
        value=('X', 'Z', 'X'),
        description='Kind:',
        style={'description_width': 'initial'}
    )
    
    ancilla_dropdown = widgets.Dropdown(
        options=[('Both', 'both'), ('X only', 'x'), ('Z only', 'z')],
        value='both',
        description='Ancilla mode:',
        style={'description_width': 'initial'}
    )
    
    edge_width_slider = widgets.FloatSlider(
        value=3.0, min=1.0, max=8.0, step=0.5,
        description='Edge width:',
        style={'description_width': 'initial'}
    )
    
    show_edges_checkbox = widgets.Checkbox(
        value=True,
        description='Show edges',
        style={'description_width': 'initial'}
    )
    
    # Output widget for the plot
    output_widget = widgets.Output()
    
    def update_plot(*args):
        """Update the plot when parameters change"""
        with output_widget:
            output_widget.clear_output(wait=True)
            try:
                fig = create_interactive_initplus(
                    d=d_slider.value,
                    kind=kind_dropdown.value,
                    ancilla_mode=ancilla_dropdown.value,
                    edge_width=edge_width_slider.value,
                    show_edges=show_edges_checkbox.value
                )
                fig.show()
            except Exception as e:
                print(f"Error creating visualization: {e}")
    
    # Connect widgets to update function
    d_slider.observe(update_plot, names='value')
    kind_dropdown.observe(update_plot, names='value')
    ancilla_dropdown.observe(update_plot, names='value')
    edge_width_slider.observe(update_plot, names='value')
    show_edges_checkbox.observe(update_plot, names='value')
    
    # Layout
    controls = widgets.VBox([
        widgets.HTML("<h3>Interactive InitPlus 3D Visualization</h3>"),
        d_slider,
        kind_dropdown,
        ancilla_dropdown,
        edge_width_slider,
        show_edges_checkbox
    ])
    
    # Create initial plot
    update_plot()
    
    return widgets.VBox([controls, output_widget])

# Display the interactive widget
interactive_widget = create_interactive_widget()
display(interactive_widget)

In [ ]:
# Simple test - create a single interactive plot
fig = create_interactive_initplus(d=3, kind=("X", "Z", "X"), ancilla_mode="both", edge_width=3.0)
fig.show()